In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the datetime module from the datetime library.
from datetime import datetime

In [4]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [5]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

772

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    time.sleep(1)
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"] 
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_wdescription =city_weather["weather"][0]["description"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_wdescription})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

| northam
Processing Record 15 of Set 7 | grand-santi
Processing Record 16 of Set 7 | cap malheureux
Processing Record 17 of Set 7 | nurota
Processing Record 18 of Set 7 | gladstone
Processing Record 19 of Set 7 | mehamn
Processing Record 20 of Set 7 | pacific grove
Processing Record 21 of Set 7 | umea
Processing Record 22 of Set 7 | great yarmouth
Processing Record 23 of Set 7 | samarai
Processing Record 24 of Set 7 | wageningen
Processing Record 25 of Set 7 | turangi
Processing Record 26 of Set 7 | petropavlovsk-kamchatskiy
Processing Record 27 of Set 7 | menongue
Processing Record 28 of Set 7 | ribeira grande
Processing Record 29 of Set 7 | ziebice
Processing Record 30 of Set 7 | cabedelo
Processing Record 31 of Set 7 | praia da vitoria
Processing Record 32 of Set 7 | avera
Processing Record 33 of Set 7 | tashtyp
Processing Record 34 of Set 7 | harur
Processing Record 35 of Set 7 | sisimiut
Processing Record 36 of Set 7 | portland
Processing Record 37 of Set 7 | talnakh
Processing R

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,48.09,74,0,8.03,clear sky
1,Atuona,PF,-9.8000,-139.0333,79.66,65,63,7.40,broken clouds
2,Mercedes,AR,-34.6515,-59.4307,59.02,46,97,7.56,overcast clouds
3,Georgetown,MY,5.4112,100.3354,80.53,88,20,1.01,few clouds
4,Mataura,NZ,-46.1927,168.8643,43.38,87,88,2.24,overcast clouds
5,Oranjestad,AW,12.5240,-70.0270,87.75,66,40,23.02,scattered clouds
6,Hithadhoo,MV,-0.6000,73.0833,83.03,74,8,13.85,clear sky
7,Srednekolymsk,RU,67.4500,153.6833,55.69,56,100,12.48,overcast clouds
8,Ushuaia,AR,-54.8000,-68.3000,38.86,81,75,0.00,broken clouds
9,Nanortalik,GL,60.1432,-45.2371,45.99,68,100,14.56,overcast clouds


In [11]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")